In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("data/labels.csv")
df.head()


,image_num,vote_1,vote_2,vote_3,vote_4,vote_5,vote_6,vote_7,vote_8,vote_9,vote_10
0,953417,0.0,0.000000,0.000000,0.040323,0.258065,0.403226,0.185484,0.080645,0.024194,0.008065
1,953777,0.0,0.023438,0.015625,0.023438,0.101562,0.312500,0.273438,0.164062,0.062500,0.023438
2,953756,0.0,0.015625,0.023438,0.070312,0.273438,0.390625,0.156250,0.039062,0.015625,0.015625
3,954195,0.0,0.008197,0.057377,0.213115,0.459016,0.188525,0.049180,0.008197,0.000000,0.016393
4,953903,0.0,0.008065,0.032258,0.040323,0.266129,0.403226,0.137097,0.072581,0.024194,0.016129


In [2]:
from utils.split_utils import load_and_split_labels

train_df, val_df, test_df = load_and_split_labels("data/labels.csv")

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
train_df.head()


Train: 184603, Val: 32578, Test: 38327


,image_num,vote_1,vote_2,vote_3,vote_4,vote_5,vote_6,vote_7,vote_8,vote_9,vote_10,mean_score
143953,684178,0.015707,0.020942,0.010471,0.083770,0.303665,0.340314,0.162304,0.036649,0.015707,0.010471,5.659686
43395,567633,0.000000,0.004902,0.014706,0.083333,0.191176,0.294118,0.210784,0.137255,0.058824,0.004902,6.259804
199848,932635,0.000000,0.007752,0.000000,0.069767,0.255814,0.387597,0.139535,0.085271,0.023256,0.031008,6.077519
200572,71307,0.007273,0.025455,0.076364,0.170909,0.254545,0.203636,0.141818,0.061818,0.029091,0.029091,5.505455
142725,836507,0.019048,0.019048,0.042857,0.119048,0.214286,0.233333,0.157143,0.128571,0.038095,0.028571,5.890476


In [3]:
from utils.split_utils import load_and_split_labels
from utils.dataset import AestheticDataset
from models.hybrid_model import HybridAestheticModel
from utils.train import train_one_epoch, evaluate

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [4]:
# Load labels with split
train_df, val_df, test_df = load_and_split_labels("data/labels.csv")

transform_train = transforms.Compose([
    transforms.Resize((320, 240)),
    transforms.RandomRotation(degrees=15),             # rotate randomly ±15 degrees
    transforms.RandomHorizontalFlip(p=0.5),             # horizontal flip with 50% chance
    transforms.ToTensor(),
])

transform = transforms.Compose([
    transforms.Resize((320, 240)),
    transforms.ToTensor(),
])

images_dir = "data/images"

train_ds = AestheticDataset(train_df, images_dir, transform_train)
val_ds = AestheticDataset(val_df, images_dir, transform)
test_ds = AestheticDataset(test_df, images_dir, transform)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=128)
test_loader = DataLoader(test_ds, batch_size=128)


In [5]:
def collect_gpu_garbage():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

In [7]:
import gc
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridAestheticModel().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
epochs = 10
train_losses, val_losses = [], []

best_val_loss = float('inf')

for epoch in range(epochs):
    collect_gpu_garbage()
    
    train_loss = train_one_epoch(model, train_loader, optimizer, device)
    
    collect_gpu_garbage()
    
    val_loss, _, _ = evaluate(model, val_loader, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

    # Save checkpoint if best val loss so far
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint_path = f"best_model_epoch_{epoch+1}_val_loss_{val_loss:.4f}.pt"
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Saved new best model checkpoint: {checkpoint_path}")


    # Free GPU memory cache


  8%|▊         | 121/1443 [23:49<4:20:22, 11.82s/it]


KeyboardInterrupt: 

In [ ]:
# Analyze your target distribution
import matplotlib.pyplot as plt
plt.hist(targets, bins=50)
plt.title('Target Score Distribution')
plt.show()

In [ ]:
# Add this to your training script
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/model_architecture')
writer.add_graph(model, (dummy_input, dummy_hsv, dummy_rgb, dummy_lab, dummy_comp))
writer.close()